`Prior` to an object `being released` from `memory`, its ***finalizer runs***,

In [ ]:
class Test
{
    ~Test() // like constructor with ~ perfix
    {
        // Finalizer logic...
    }
}

***finalizers*** `cannot` be declared as **public** or **static**, `cannot` **have parameters**, and `cannot` **call the base class**

<div dir="rtl" style="margin:auto; width:80%;">
<h3>فازهای جمع‌آوری زباله</h3>
<p>جمع‌آوری زباله در چند فاز مجزا انجام می‌شود:</p>
<ol><li><p><strong>شناسایی اشیاء بلااستفاده:</strong></p><ul><li>در این فاز، GC اشیائی را که دیگر مورد استفاده نیستند و باید حذف شوند، شناسایی می‌کند.</li></ul></li><li><p><strong>حذف اشیاء بدون نهایی‌ساز:</strong></p><ul><li>اشیائی که نهایی‌ساز (finalizer) ندارند، بلافاصله حذف می‌شوند.</li></ul></li><li><p><strong>اشیاء با نهایی‌سازهای در حال انتظار:</strong></p><ul><li>اشیائی که نهایی‌ساز دارند، در این مرحله هنوز حذف نمی‌شوند. این اشیاء به یک صف ویژه (special queue) اضافه می‌شوند.</li></ul></li></ol>
<h3>ادامه اجرای برنامه</h3>
<p>بعد از اینکه جمع‌آوری زباله به این مرحله رسید، عملیات جمع‌آوری زباله به پایان می‌رسد و برنامه شما به اجرای خود ادامه می‌دهد. سپس یک نخ (thread) ویژه برای اجرای نهایی‌سازها فعال می‌شود.</p>
<h3>اجرای نهایی‌سازها</h3>
<p>نخ نهایی‌ساز در کنار برنامه شما شروع به کار می‌کند. این نخ اشیاء را از صف ویژه خارج می‌کند و متدهای نهایی‌ساز آنها را اجرا می‌کند.</p>
<ol><li><p><strong>نگه داشتن اشیاء زنده تا زمان اجرای نهایی‌ساز:</strong></p><ul><li>قبل از اجرای نهایی‌ساز، شیء هنوز زنده است، زیرا صف ویژه به عنوان یک ریشه (root) عمل می‌کند و مانع از حذف شیء می‌شود.</li></ul></li><li><p><strong>اجرای نهایی‌ساز:</strong></p><ul><li>بعد از اجرای نهایی‌ساز، شیء دیگر زنده نخواهد بود و به عنوان یک شیء بلااستفاده (orphaned) در نظر گرفته می‌شود.</li></ul></li><li><p><strong>حذف در جمع‌آوری زباله بعدی:</strong></p><ul><li>در جمع‌آوری زباله بعدی برای نسل آن شیء، شیء حذف می‌شود.</li></ul></li></ol>
</div>

Here are some guidelines for implementing finalizers:  
- Ensure that your finalizer `executes quickly`.
- Never `block` in your finalizer (see “Blocking” on page 612).
- `Don’t reference` other finalizable objects.
- `Don’t throw exceptions`.

<div dir="rtl" style="width:90%; margin:auto">
<h3>منابع مدیریت شده (Managed Resources)</h3>
<p>این منابع توسط محیط زمان اجرای دات‌نت (CLR - Common Language Runtime) مدیریت می‌شوند. این منابع شامل اشیاءی هستند که توسط زبان‌های دات‌نت مانند C# ایجاد شده‌اند و توسط سیستم جمع‌آوری زباله (Garbage Collector) به صورت خودکار مدیریت و آزاد می‌شوند.</p>

<ul><li><strong>مثال‌ها</strong>:<ul><li>اشیاء دات‌نت (مانند رشته‌ها، لیست‌ها، دیکشنری‌ها، ...)</li><li>اشیاءی که از رابط IDisposable پیروی می‌کنند (مانند <code>FileStream</code>, <code>StreamReader</code>, <code>SqlConnection</code> و غیره).</li></ul></li></ul>

<h3>منابع مدیریت نشده (Unmanaged Resources)</h3>
<p>این منابع توسط CLR مدیریت نمی‌شوند و شامل منابعی هستند که به صورت مستقیم با سیستم عامل تعامل دارند. این منابع نیاز به مدیریت صریح دارند تا از نشت حافظه جلوگیری شود.</p>
<ul><li><strong>مثال‌ها</strong>:<ul><li>دسته‌های فایل (File Handles)</li><li>دسته‌های ویندوز (Window Handles)</li><li>حافظه اختصاص داده شده به صورت مستقیم توسط فراخوانی‌های سیستم عامل (مانند <code>malloc</code> در زبان‌های C/C++)</li><li>سوکت‌های شبکه</li></ul></li></ul>
</div>

<div dir="rtl" style="width:90%; margin:auto">
<h3>نتیجه گیری</h3>
<p>منابع مدیریت شده توسط سیستم جمع‌آوری زباله (Garbage Collector) به صورت خودکار مدیریت و آزاد می‌شوند.</p>

<p>منابع مدیریت نشده، توسط CLR مدیریت نمی‌شوند،  این منابع نیاز به مدیریت صریح دارند تا از نشت حافظه جلوگیری شود</p>

<p>پس در مکانیزمی که طراحی می کنیم باید حتما منابع مدیریت نشده آزاد شوند و بنا به تصمیم ما میتوانیم منابع مدیریت شده را خودمان dispose کنیم، یا اینکه بگذاریم GC این کار را انجام دهد.</p>

<div>

### Calling Dispose from a Finalizer

A popular pattern is to have the ***finalizer*** `call Dispose`.

In [2]:
class Test : IDisposable
{
    public void Dispose() // از بیرون قابلیت فراخوانی دارد.
    {
        Dispose (true); // میخواهیم علاوه بر منابع مدیریت نشده ، منابع مدیریت شده هم 
        // آزاد شوند
        GC.SuppressFinalize (this); // به GC اعلام می کنیم ، تو نیاز نیست 
        // finalizer این شیء را فراخوانی نکن، ما خودمان به صورت دستی این کار را انجام
        // دادیم. چون اجرای finalizer هزینه زیادی دارد
    }
    protected virtual void Dispose (bool disposing)
    {
        if (disposing)//برای آزاد سازی منابع مدیریت نشده
        {
            // Call Dispose() on other objects owned by this instance.
            // You can reference other finalizable objects here.
            // ...
        }
        // Release unmanaged resources owned by (just) this object.
        // ...
    }
    ~Test() => Dispose (false); // در صورتی که Dispose عمومی فراخوانی نشد، 
    // منابع مدیریت نشده حتما آزاد شوند.
}

In [ ]:
using System.IO;

class MyClass : IDisposable
{
    private FileStream fileStream; // Managed resource
    private IntPtr fileHandle; // Unmanaged resource

    public MyClass(string fileName)
    {
        fileStream = new FileStream(fileName, FileMode.Open);
        fileHandle = fileStream.SafeFileHandle.DangerousGetHandle();
    }

    public void Dispose()
    {
        Dispose(true);
        GC.SuppressFinalize(this); // جلوگیری از اجرای نهایی‌کننده
    }

    protected virtual void Dispose(bool disposing)
    {
        if (disposing)
        {
            // آزادسازی منابع مدیریت شده
            if (fileStream != null)
            {
                fileStream.Dispose();
                fileStream = null;
            }
        }

        // آزادسازی منابع مدیریت نشده
        if (fileHandle != IntPtr.Zero)
        {
            // Close the handle using Win32 API
            CloseHandle(fileHandle);
            fileHandle = IntPtr.Zero;
        }
    }

    ~MyClass()
    {
        Dispose(false);
    }

    [System.Runtime.InteropServices.DllImport("kernel32.dll", SetLastError = true)]
    private static extern bool CloseHandle(IntPtr hObject);
}


### Resurrection (تجدید حیات)

<div dir="rtl" style="width:90%; margin:auto;">
نهایی‌کننده (Finalizer) یک شیء در حال نابود شدن (Dying Object) را طوری تغییر دهد که دوباره به یک شیء زنده ارجاع دهد. این باعث می‌شود که شیء در حال نابود شدن دوباره به حالت زنده برگردد و جمع‌آوری زباله نتواند آن را پاک کند.
</div>

In [ ]:
using System;
using System.IO;

class TempFileManager : IDisposable
{
    private string filePath;
    private static TempFileManager resurrectedInstance;
    
    public TempFileManager(string path)
    {
        filePath = path;
        File.Create(filePath).Dispose(); // Create the temporary file
    }

    public void Dispose()
    {
        Dispose(true);
        GC.SuppressFinalize(this); // جلوگیری از اجرای نهایی‌کننده
    }

    protected virtual void Dispose(bool disposing)
    {
        if (disposing)
        {
            // آزادسازی منابع مدیریت شده
        }

        // حذف فایل موقت
        if (File.Exists(filePath))
        {
            File.Delete(filePath);
        }
    }

    ~TempFileManager()
    {
        // تجدید حیات: دوباره به شیء زنده ارجاع داده می‌شود
        resurrectedInstance = this;
    }
}

class Program
{
    static void Main(string[] args)
    {
        TempFileManager tempFile = new TempFileManager("temp.txt");

        // حذف مرجع به شیء
        tempFile = null;

        // اجرا کردن جمع‌آوری زباله
        GC.Collect();
        GC.WaitForPendingFinalizers();

        // شیء دوباره زنده شده و فایل موقت هنوز وجود دارد
        Console.WriteLine(File.Exists("temp.txt")); // True
    }
}


### GC.ReRegisterForFinalize

<p dir="rtl">
اگر یک شیء پس از فراخوانی نهایی‌کننده (Finalizer) دوباره زنده (Resurrected) شود، نهایی‌کننده آن به صورت پیش‌فرض دوباره فراخوانی نخواهد شد، مگر اینکه به صورت صریح با استفاده از GC.ReRegisterForFinalize ثبت شود تا دوباره در چرخه نهایی‌سازی قرار گیرد.
</p>

In [ ]:
using System;
using System.IO;

class TempFileManager : IDisposable
{
    private string filePath;
    private static TempFileManager resurrectedInstance;
    
    public TempFileManager(string path)
    {
        filePath = path;
        File.Create(filePath).Dispose(); // Create the temporary file
    }

    public void Dispose()
    {
        Dispose(true);
        GC.SuppressFinalize(this); // جلوگیری از اجرای نهایی‌کننده
    }

    protected virtual void Dispose(bool disposing)
    {
        if (disposing)
        {
            // آزادسازی منابع مدیریت شده
        }

        // حذف فایل موقت
        if (File.Exists(filePath))
        {
            File.Delete(filePath);
        }
    }

    ~TempFileManager()
    {
        Console.WriteLine("Finalizer called");

        // تجدید حیات: دوباره به شیء زنده ارجاع داده می‌شود
        resurrectedInstance = this;

        // ثبت دوباره برای نهایی‌سازی
        GC.ReRegisterForFinalize(this);
    }
}

class Program
{
    static void Main(string[] args)
    {
        TempFileManager tempFile = new TempFileManager("temp.txt");

        // حذف مرجع به شیء
        tempFile = null;

        // اجرا کردن جمع‌آوری زباله
        GC.Collect();
        GC.WaitForPendingFinalizers();

        // شیء دوباره زنده شده و فایل موقت هنوز وجود دارد
        Console.WriteLine(File.Exists("temp.txt")); // True

        // حذف مرجع به شیء دوباره زنده شده
        tempFile.resurrectedInstance = null;

        // اجرا کردن جمع‌آوری زباله مجدد
        GC.Collect();
        GC.WaitForPendingFinalizers();

        // اکنون فایل موقت باید حذف شده باشد
        Console.WriteLine(File.Exists("temp.txt")); // False
    }
}
